# Desafio 1 - Ciência e Governança de Dados
## Universidade Federal de Lavras  

### Autor: **Paulo Henrique dos Anjos Silveira** - 202310533  

---

## Descrição

O presente trabalho tem como objetivo entender como fatores econômicos, sociais e estruturais se relacionam e impactam o desenvolvimento socioeconômico nos municípios da Região Sudeste do Brasil, conforme proposto no Desafio I – Ciência e Governança de Dados da Zetta Lab/UFLA

Foram utilizadas bases oficiais do IBGE, DATASUS, Atlas do Desenvolvimento Humano e ANATEL, com indicadores recentes de:
- População Residente (2022)
- PIB Municipal a preços de mercado (2021)
- Taxa de Desemprego média (2022)
- Índice Brasileiro de Conectividade – IBC (2022)
- Número de Homicídios (2022)
- Expectativa de Vida ao Nascer (2010)
- IDHM (2021)
- Taxa de Analfabetismo (2022)

Após a limpeza e padronização dos dados, as tabelas foram unificadas e agregadas por estado. Em seguida, aplicaram-se análises de correlação e regressão linear para identificar os fatores com maior associação ao desenvolvimento humano e econômico.

# Importando biblitecas necessárias

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, mean_squared_error

color_sequence = px.colors.qualitative.Set2

## Indicadores utilizados

Os dados utilizados neste trabalho foram selecionados dentro de um bienio (2021–2022), com o objetivo de garantir maior consistência temporal e comparabilidade entre os indicadores socioeconômicos.
A escolha de um intervalo curto reduz distorções causadas por variações de metodologia e mudanças conjunturais.

Entretanto, o indicador de Expectativa de Vida ao Nascer teve como referência o censo de 2010, pois é a última base oficial consolidada pelo IBGE com abrangência nacional e municipal.

## Bases de dados

### População residente - total sudeste
- Fonte: Instituto Brasileiro de Geografia e Estatística
- Frequência: Ano 2022
- Unidade: Habitante
- Houve tratamento de dados para este indicador? Sim

> [Informação]
> Tratamento de dados: renomeação de coluna, remoção de coluna vazia, remoção de nulos e tratamento de vazios.

In [3]:
populacao = pd.read_csv('dados/populacao_total.csv', sep=';')

# Renomeia coluna de população
populacao = populacao.rename(columns={'2022': 'populacao'})

# Remove colunas vazias e converte tipo
populacao = populacao.drop(columns=['Código'])
populacao = populacao.dropna(axis=1, how='all')
populacao['populacao'] = pd.to_numeric(populacao['populacao'], errors='coerce')

# Preenche nulos com a média por estado (Sigla) e converte para inteiro
populacao['populacao'] = (
    populacao.groupby('Sigla')['populacao']
      .transform(lambda x: x.fillna(round(x.mean())))
      .astype(int)
)

fig = px.scatter(
    populacao,
    x=np.arange(len(populacao)),
    y='populacao',
    hover_name='Município',
    hover_data={'Sigla': True, 'populacao': True},
    color='Sigla',
    color_discrete_sequence=color_sequence,
    title='População Total por Município (2022)'
)
fig.update_layout(
    xaxis_title='Municípios',
    yaxis_title='População (escala log)',
    xaxis=dict(showticklabels=False),
    yaxis_type='log'
)
fig.show()



### PIB Municipal - preços de mercado (preços de 2010)
- Fonte: Instituto Brasileiro de Geografia e Estatística
- Frequência: Ano 2021
- Unidade: R$ (mil), a preços do ano 2010
- Houve tratamento de dados para este indicador? Sim

> [Informação]
> Tratamento de dados: remoção de coluna vazia e remoção de nulos.

In [4]:
pib = pd.read_csv('dados/pib_precos_de_mercado.csv', sep=';', decimal=',')

# Renomeia coluna de pib
pib = pib.rename(columns={'2021': 'pib'})

# Remove colunas vazias e converte tipo
pib = pib.drop(columns=['Código'])
pib = pib.dropna(axis=1, how='all')
pib['pib'] = pd.to_numeric(pib['pib'], errors='coerce')

# Preenche nulos com a média por estado (Sigla) e converte para inteiro
pib['pib'] = (
    pib.groupby('Sigla')['pib']
      .transform(lambda x: x.fillna(round(x.mean())))
      .astype(int)
)

fig = px.scatter(
    pib,
    x=np.arange(len(pib)),
    y='pib',
    hover_name='Município',
    hover_data={'Sigla': True, 'pib': True},
    color='Sigla',
    color_discrete_sequence=color_sequence,
    title='PIB a Preços de Mercado (2021)'
)
fig.update_layout(
    xaxis_title='Municípios',
    yaxis_title='PIB (R$) - log',
    yaxis_type='log',
    xaxis=dict(showticklabels=False)
)
fig.show()

### Taxa de desemprego
- Fonte: Instituto Brasileiro de Geografia e Estatística, Pesquisa Nacional por Amostra de Domicílios Contínua (IBGE/PNAD Contínua)
- Frequência: Trimestral de 2022 T1 até 2025 T4
- Unidade: (%)
- Houve tratamento de dados para este indicador? Sim

> [Informação]
> Tratamento de dados: remoção de coluna vazia e coluna desnecessária.

In [5]:
desemprego = pd.read_csv('dados/taxa_de_desemprego.csv', sep=';', decimal=',')

# Renomeia coluna de desemprego
desemprego = desemprego.rename(columns={'2022': 'desemprego'})

# Calcular a média das colunas trimestrais
desemprego['desemprego'] = desemprego[['2022 T1', '2022 T2', '2022 T3', '2022 T4']].mean(axis=1).round(1)

# Remove colunas vazias
desemprego = desemprego.drop(columns=['Código','Estado','2022 T1', '2022 T2', '2022 T3', '2022 T4'])
desemprego = desemprego.dropna(axis=1, how='all')

fig = px.bar(
    desemprego,
    x='Sigla',
    y='desemprego',
    color='Sigla',
    color_discrete_sequence=color_sequence,
    title='Evolução da Taxa de Desemprego por Estado'
)
fig.update_layout(yaxis_title='Taxa (%)')
fig.show()

### Índice Brasileiro de Conectividade
- Fonte: Agência Nacional de Telecomunicações
- Frequência: Ano 2024
- Unidade: Percentil
- Houve tratamento de dados para este indicador? Não

> [Informação]
> Tratamento de dados: filtração da região sudeste e do ano 2022, remoção de sufixo para texto padronizado e coluna desnecessária

In [16]:
internet = pd.read_csv('dados/cobertura_internet.csv', sep=';', decimal=',')

# Lista de UFs da região Sudeste
sudeste = ['SP', 'RJ', 'MG', 'ES']

# Filtra apenas linhas da região Sudeste e do ano 2022
internet_sudeste = internet[
    (internet['UF'].isin(sudeste)) &
    (internet['Ano'] == 2022)
].copy()

# Remove o sufixo " - XX" do nome do município
internet_sudeste['Município'] = internet_sudeste['Município'].str.replace(r' - [A-Z]{2}$', '', regex=True)

# Remove colunas desnecessárias
internet_sudeste = internet_sudeste.drop(columns=['Código Município','Ano','Cobertura Pop. 4G5G', 'Densidade SMP', 'HHI SMP', 'Densidade SCM', 'HHI SCM', 'Adensamento Estações', 'Fibra', 'Cobertura área agricultável'])

# Renomeia colunas
internet_sudeste = internet_sudeste.rename(columns={'UF': 'Sigla', 'IBC': 'internet'})

fig = px.scatter(
    internet_sudeste,
    x=np.arange(len(internet_sudeste)),
    y='internet',
    hover_name='Município',
    hover_data={'Sigla': True, 'internet': True},
    color='Sigla',
    color_discrete_sequence=color_sequence,
    title='Cobertura de Internet 4G - Região Sudeste (2022)'
)
fig.update_layout(
    xaxis_title='Municípios',
    yaxis_title='Cobertura (%) - log',
    yaxis_type='log',
    xaxis=dict(showticklabels=False)
)
fig.show()

### Número de homicídios
- Fonte: DATASUS – Site SUS (Sistema Único de Saúde).
- Frequência: Ano 2022
- Unidade: Unidade
- Houve tratamento de dados para este indicador?

> [Informação]
> Tratamento de dados: remoção de coluna vazia e remoção de nulos.

In [7]:
homicidios = pd.read_csv('dados/numero_homicidios.csv', sep=';', decimal=',')

# Renomeia coluna de homicidios
homicidios = homicidios.rename(columns={'2022': 'homicidios'})

# Remove colunas vazias e converte tipo
homicidios = homicidios.drop(columns=['Código'])
homicidios = homicidios.dropna(axis=1, how='all')
homicidios['homicidios'] = pd.to_numeric(homicidios['homicidios'], errors='coerce')

# Preenche nulos e transforma em inteiro
homicidios['homicidios'] = homicidios['homicidios'].fillna(0).astype(int)

fig = px.scatter(
    homicidios,
    x=np.arange(len(homicidios)),
    y='homicidios',
    hover_name='Município',
    hover_data={'Sigla': True, 'homicidios': True},
    color='Sigla',
    color_discrete_sequence=color_sequence,
    title='Número de Homicídios por Município (2022) - Escala Logarítmica'
)
fig.update_layout(
    xaxis_title='Municípios',
    yaxis_title='Homicídios (log)',
    yaxis_type='log',
    xaxis=dict(showticklabels=False)
)
fig.show()

### Estimativa da esperança de vida ao nascer
Frequência: Decenal

Fonte: Instituto Brasileiro de Geografia e Estatística, Departamento de População e Indicadores Sociais. Divisão de Estudos e Análises da Dinâmica Demográfica (IBGE/Pop)

Unidade: Anos

> [Informação]
> Tratamento de dados: remoção de colunas desnecessárias, coluna vazia e remoção de nulos.

In [8]:
expectativa_de_vida = pd.read_csv('dados/esperança_de_vida_ao_nascer.csv', sep=';', decimal=',')

# Renomeia coluna de expectativa_de_vida
expectativa_de_vida = expectativa_de_vida.rename(columns={'2010': 'expectativa'})

# Remove colunas desnecessárias
expectativa_de_vida = expectativa_de_vida.drop(columns=['Código','1970', '1980', '1991', '2000'])

# Remove colunas vazias e converte tipo
expectativa_de_vida = expectativa_de_vida.dropna(axis=1, how='all')
expectativa_de_vida['expectativa'] = pd.to_numeric(expectativa_de_vida['expectativa'], errors='coerce')

# Preenche nulos com a média por estado (Sigla) e converte para inteiro
expectativa_de_vida['expectativa'] = (
    expectativa_de_vida.groupby('Sigla')['expectativa']
      .transform(lambda x: x.fillna(x.mean()))
      .round(2)
)

fig = px.scatter(
    expectativa_de_vida,
    x=np.arange(len(expectativa_de_vida)),
    y='expectativa',
    hover_name='Município',
    hover_data={'Sigla': True, 'expectativa': True},
    color='Sigla',
    color_discrete_sequence=color_sequence,
    title='Expectativa de Vida ao Nascer (2010)'
)
fig.update_layout(
    xaxis_title='Municípios',
    yaxis_title='Expectativa de Vida (anos) - log',
    yaxis_type='log',
    xaxis=dict(showticklabels=False)
)
fig.show()

### IDHM - Índice de Desenvolvimento Humano Municipal (PNUD) - Brasil

Fonte: Programa das Nações Unidas para Desenvolvimento

Frequência: Ano 2023

Unidade: Percentil de 0 a 1

> [Informação]
> Tratamento de dados: remoção de colunas desnecessárias, coluna vazia e remoção de nulos.

In [9]:
idhm_estado = pd.read_csv('dados/idhm_estados.csv', sep=';', decimal=',')

# Renomeia coluna de IDHM
idhm_estado = idhm_estado.rename(columns={'2021': 'idhm'})

# Remove colunas desnecessárias
idhm_estado = idhm_estado.drop(columns=['Código','Estado','2017', '2018', '2019', '2020'])

# Remove colunas vazias e converte tipo
idhm_estado = idhm_estado.dropna(axis=1, how='all')
idhm_estado['idhm'] = pd.to_numeric(idhm_estado['idhm'], errors='coerce')

# Preenche nulos com a média por estado (Sigla) e converte para inteiro
idhm_estado['idhm'] = (
    idhm_estado.groupby('Sigla')['idhm']
      .transform(lambda x: x.fillna(x.mean()))
      .round(3)
)


fig = px.bar(
    idhm_estado,
    x='Sigla',
    y='idhm',
    color='Sigla',
    color_discrete_sequence=color_sequence,
    title='Índice de Desenvolvimento Humano por Estado (2021)'
)
fig.update_layout(xaxis_title='Estado', yaxis_title='IDHM')
fig.show()

### Taxa de analfabetismo - pessoas com 15 anos ou mais (Atlas DH - Censo)

Fonte: Atlas do Desenvolvimento Humano (Censo Demográfico)

Frequência: Ano 2022

Unidade: (%)

In [10]:
analfabetismo = pd.read_csv('dados/taxa_de_analfabetismo.csv', sep=';', decimal=',')

# Renomeia coluna de analfabetismo
analfabetismo = analfabetismo.rename(columns={'2022': 'analfabetismo'})

# Remove colunas desnecessárias
analfabetismo = analfabetismo.drop(columns=['Código'])

# Remove colunas vazias e converte tipo
analfabetismo = analfabetismo.dropna(axis=1, how='all')
analfabetismo['analfabetismo'] = pd.to_numeric(analfabetismo['analfabetismo'], errors='coerce')

# Preenche nulos com a média por estado (Sigla) e converte para inteiro
analfabetismo['analfabetismo'] = (
    analfabetismo.groupby('Sigla')['analfabetismo']
      .transform(lambda x: x.fillna(x.mean()))
      .round(3)
)

fig = px.scatter(
    analfabetismo,
    x=np.arange(len(analfabetismo)),
    y='analfabetismo',
    hover_name='Município',
    hover_data={'Sigla': True, 'analfabetismo': True},
    color='Sigla',
    color_discrete_sequence=color_sequence,
    title='Taxa de Analfabetismo (2022)'
)
fig.update_layout(
    xaxis_title='Municípios',
    yaxis_title='Taxa de Analfabetismo (%) - log',
    yaxis_type='log',
    xaxis=dict(showticklabels=False)
)
fig.show()

## Unindo as bases de dados em uma tabela única
### Preparação dos dados


In [11]:
# Iniciar com população
df = populacao.merge(pib, on=['Sigla', 'Município'], how='inner')
df = df.merge(internet_sudeste, on=['Sigla', 'Município'], how='inner')
df = df.merge(homicidios, on=['Sigla', 'Município'], how='inner')
df = df.merge(expectativa_de_vida, on=['Sigla', 'Município'], how='inner')
df = df.merge(idhm_estado, on=['Sigla'], how='inner')
df = df.merge(desemprego, on=['Sigla'], how='inner')
df = df.merge(analfabetismo, on=['Sigla', 'Município'], how='inner')

internet.rename(columns={'UF': 'Sigla'}, inplace=True)

# Padronizar nomes das colunas
df.rename(columns={
    'Sigla': 'sigla',
    'Município': 'municipio'
}, inplace=True)

# display(df.head())

### Correlacionando os dados 

In [12]:
df_estado = df.groupby('sigla').agg({
    'municipio': 'count',
    'populacao': 'sum',
    'pib': 'sum',
    'internet': 'mean',
    'homicidios': 'mean',
    'expectativa': 'mean',
    'idhm': 'mean',
    'desemprego': 'mean',
    'analfabetismo': 'mean'
}).reset_index()

display(df_estado.head())


,sigla,municipio,populacao,pib,internet,homicidios,expectativa,idhm,desemprego,analfabetismo
0,ES,78,3833712,86983201,57.436667,14.410256,74.536026,0.771,7.9,8.594872
1,MG,851,20515381,400174209,51.473384,3.138660,74.422538,0.774,7.2,9.636193
2,RJ,90,15999629,442114743,65.615667,40.400000,73.944111,0.762,12.8,5.604000
3,SP,642,43616484,1250818403,63.977710,4.788162,75.148255,0.806,9.1,5.155514


## MATRIZ DE CORRELAÇÃO

A matriz de correlação apresenta a força e o sentido das relações entre os indicadores socioeconômicos analisados.
O coeficiente de correlação de Pearson (ρ) varia entre –1 e +1, indicando:
    
    - +1: correlação positiva perfeita (as variáveis crescem juntas);

    - 0: ausência de correlação linear;

    - –1: correlação negativa perfeita (quando uma cresce, a outra diminui).

Neste estudo, a matriz permitiu identificar quais fatores estão mais associados ao desenvolvimento socioeconômico dos estados do Sudeste.

In [13]:
cols = ['populacao','internet','pib','homicidios','expectativa','idhm','desemprego']
corr = df_estado[cols].corr().round(2)

fig_corr = px.imshow(
    corr,
    text_auto=True,
    color_continuous_scale='RdBu',
    zmin=-1, zmax=1,
    title='Correlação entre Indicadores por Estado - Região Sudeste'
)
fig_corr.update_layout(
    template='simple_white',
    coloraxis_colorbar=dict(title='ρ')
)
fig_corr.show()

## APLICANDO REGRESSÃO NO MODELO

### Por que usar regressão (e não apenas correlação)?

- A correlação mede apenas o grau de associação linear entre duas variáveis, sem indicar direção nem efeito.
Já a regressão busca estimar a relação funcional entre uma variável explicativa (x) e uma resposta (y).

In [14]:
pares = [
    ('populacao', 'pib'),
    ('pib', 'expectativa'),
    ('internet', 'idhm'),
    ('homicidios', 'expectativa'),
    ('idhm', 'desemprego'),
    ('analfabetismo', 'idhm'),
    ('internet', 'analfabetismo')
]

# rótulos para os eixos/legenda
labels = {
    'sigla': 'Estado',
    'municipio': 'Nº de Municípios',
    'populacao': 'População',
    'pib': 'PIB (R$)',
    'internet': 'Cobertura Média de Internet (%)',
    'homicidios': 'Homicídios (média)',
    'expectativa': 'Expectativa de Vida (anos)',
    'idhm': 'IDHM',
    'desemprego': 'Desemprego (%)',
    'analfabetismo': 'Analfabetismo (%)'
}


### Plotando os gráficos
- cada gráfico exibe a regressão no título.
- Destaque para cada estado.
- cada gráfico recebe métricas.

### Principais métricas de regressão
- R² (Coeficiente de determinação)
    - Mede o quanto da variação de y é explicada por x. Vai de 0 a 1 (quanto maior, melhor o ajuste).
- MAE — Mean Absolute Error (Erro absoluto médio)
    - Mede o erro médio em valor absoluto (sem penalizar erros grandes).
- MSE — Mean Squared Error (Erro quadrático médio)
    - Mede o erro médio, mas penalizando mais fortemente os erros grandes (por elevar ao quadrado) quanto menor, melhor.

In [ ]:
for x, y in pares:
    #  Ajuste do modelo OLS 
    X = sm.add_constant(df_estado[x])
    y_data = df_estado[y]
    model = sm.OLS(y_data, X).fit()

    # Predição (para desenhar a linha)
    x_pred = np.linspace(df_estado[x].min(), df_estado[x].max(), 100)
    y_pred = model.predict(sm.add_constant(x_pred))

    #  Cálculo de métricas 
    y_fitted = model.predict(X)
    r2 = model.rsquared
    mae = mean_absolute_error(y_data, y_fitted)
    mse = mean_squared_error(y_data, y_fitted)

    # Gráfico de dispersão
    fig = px.scatter(
        df_estado,
        x=x,
        y=y,
        color='sigla',
        color_discrete_sequence=color_sequence,
        hover_name='sigla',
        text='sigla',
        labels=labels,
        title=(
            f"{labels.get(x,x)} × {labels.get(y,y)}<br>"
            f"<sup>R² = {r2:.3f} | MAE = {mae:.2f} | MSE = {mse:.2f}</sup>"
        )
    )
    print(f'Par: {x} x {y} | R²: {r2:.3f} | MAE: {mae:.2f} | MSE: {mse:.2f}')
    # Linha de regressão
    fig.add_trace(go.Scatter(
        x=x_pred,
        y=y_pred,
        mode='lines',
        name='Tendência (OLS)',
        line=dict(color='red', width=3)
    ))

    # Ajustes visuais
    fig.update_traces(textposition='top center', marker=dict(size=14, opacity=0.85))
    fig.update_layout(template='simple_white')

    # Exibir gráfico
    fig.show()

## Análises e Resultados

As principais correlações obtidas foram:

| Relação | Correlação | Interpretação |
|----------|-------------|----------------|
| **PIB × IDHM** | **+0.881** | Estados com maior PIB apresentam IDHM mais elevado, reforçando a relação entre economia e desenvolvimento humano. |
| **PIB × Expectativa de vida** | **+0.950** | Alta associação positiva: riqueza correlaciona-se com longevidade. |
| **Internet × IDHM** | **+0.198** | Relação positiva, porém fraca — o IDHM de 2010 ainda não reflete os ganhos recentes de conectividade. |
| **Homicídios × Expectativa de vida** | **–0.647** | Relação negativa moderada: maior violência implica menor longevidade. |
| **Desemprego × PIB** | **–0.998** | Correlação inversa muito forte: estados com maior PIB possuem menor taxa média de desemprego. |

A regressão linear demonstrou ajustes consistentes (R² elevados) nas relações entre PIB e Expectativa de Vida, reforçando a dependência direta entre condições econômicas e bem-estar populacional.
Por outro lado, o baixo coeficiente entre Internet e IDHM sugere defasagem temporal entre os indicadores — o IDHM ainda não incorpora totalmente o impacto da expansão digital recente.

## Conclusão

Os resultados evidenciam que, na Região Sudeste, o PIB, o IDHM e a Expectativa de Vida são os principais determinantes do desenvolvimento socioeconômico, enquanto violência e desemprego se destacam como fatores limitantes.

O estudo confirma que:
- Estados mais ricos apresentam melhores condições de vida e menor vulnerabilidade social;
- Conectividade digital desponta como nova dimensão de desigualdade, embora ainda sub-representada em indicadores tradicionais;
- Homicídios e desemprego estão fortemente associados à redução da longevidade e da qualidade de vida.